## Environment preparation :

In [ ]:
%tensorflow_version 1.x  #Select module of the tensorflow
!pip show tensorflow
import tensorflow as tf

tf.test.is_gpu_available()


`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x  #Select module of the tensorflow`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Name: tensorflow
Version: 1.15.2
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /tensorflow-1.15.2/python2.7
Requires: gast, grpcio, numpy, wheel, tensorboard, protobuf, keras-preprocessing, backports.weakref, tensorflow-estimator, keras-applications, functools32, wrapt, six, absl-py, opt-einsum, termcolor, google-pasta, enum34, mock, astor
Required-by: magenta, fancyimpute


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Data Preparation for the first training:

In [ ]:
import os
import shutil
shutil.rmtree('/content/dataset', ignore_errors=True)
!unzip -qq '/content/drive/My Drive/NYU.zip' -d /content/

In [ ]:
import os
import shutil
os.chdir('/')
%cd '/content/drive/My Drive/CrossinfoNet'

/content/drive/My Drive/CrossinfoNet


In [ ]:
import sys
#sys.path.append('../../')#add root directory
from data.importers import NYUImporter
from util.handdetector import HandDetector
from data.transformations import transformPoints2D

import numpy as np
from util.preprocess import augmentCrop,norm_dm,joints_heatmap_gen
import tensorflow as tf
rng=np.random.RandomState(23455)

from netlib.basemodel import basenet2


W0904 14:35:24.494524 139899675928448 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
train_root='/content/dataset'
di = NYUImporter(train_root, useCache=False, cacheDir='/content/handpose/cache/NYU/',refineNet=None,allJoints=False)

In [ ]:
Seq_test = di.loadSequence('test', rng=rng, shuffle=True, docom=False,cube=None)
test_num=len(Seq_test.data)

/content/dataset/test/joint_data.mat


Loading testN/A%|                                                             |

******
Loading test


Loading test100%|#############################################################|


Loaded 8252 samples.
Shuffling


In [ ]:
FileNames = []
Xmins     = []
Xmaxs     = []
Ymins     = []
Ymaxs     = []
Widths    = []
Heights   = []
Classes   = []
for i in range(test_num):
  FileNames.append(Seq_test.data[i][0].split('/')[-1])
  Xmins.append(Seq_test.data[i][1][0])
  Xmaxs.append(Seq_test.data[i][1][1])
  Ymins.append(Seq_test.data[i][1][2])
  Ymaxs.append(Seq_test.data[i][1][3])
  Widths.append(640)
  Heights.append(480)
  Classes.append('hand')

In [ ]:
import pandas as pd
DF_test = pd.DataFrame({'filename':FileNames,'width':Widths,'height':Heights,'class':Classes,'xmin':Xmins,'ymin':Ymins,'xmax':Xmaxs,'ymax':Ymaxs})

DF_test.head()

,class,filename,height,width,xmax,xmin,ymax,ymin
0,hand,depth_1_0003306.png,480,640,475,290,433,247
1,hand,depth_1_0001386.png,480,640,372,155,367,150
2,hand,depth_1_0000145.png,480,640,310,103,366,159
3,hand,depth_1_0000617.png,480,640,369,127,440,198
4,hand,depth_1_0007036.png,480,640,377,215,365,203


In [ ]:
DF_test.to_csv('CSV_TFRecords_test.csv', encoding='utf-8', index=False)


In [ ]:
Seq_train = di.loadSequence('train', rng=rng, shuffle=True, docom=False,cube=None)
train_num=len(Seq_train.data)

/content/dataset/train/joint_data.mat


Loading trainN/A%|                                                            |

******
Loading train


Loading train100%|############################################################|


Loaded 72757 samples.
Shuffling


In [ ]:
FileNames = []
Xmins     = []
Xmaxs     = []
Ymins     = []
Ymaxs     = []
Widths    = []
Heights   = []
Classes   = []
for i in range(train_num):
  FileNames.append(Seq_train.data[i][0].split('/')[-1])
  Xmins.append(Seq_train.data[i][1][0])
  Xmaxs.append(Seq_train.data[i][1][1])
  Ymins.append(Seq_train.data[i][1][2])
  Ymaxs.append(Seq_train.data[i][1][3])
  Widths.append(640)
  Heights.append(480)
  Classes.append('hand')

In [ ]:
import pandas as pd
DF_train = pd.DataFrame({'filename':FileNames,'width':Widths,'height':Heights,'class':Classes,'xmin':Xmins,'ymin':Ymins,'xmax':Xmaxs,'ymax':Ymaxs})

DF_train.head()

,class,filename,height,width,xmax,xmin,ymax,ymin
0,hand,depth_1_0055674.png,480,640,460,208,429,178
1,hand,depth_1_0027484.png,480,640,279,44,335,100
2,hand,depth_1_0057513.png,480,640,408,160,427,179
3,hand,depth_1_0002287.png,480,640,312,120,335,144
4,hand,depth_1_0037388.png,480,640,362,124,400,163


In [ ]:
DF_train.to_csv('CSV_TFRecords_train.csv', encoding='utf-8', index=False)